<!DOCTYPE html>
<html lang="pt">
<head>
    <meta charset="UTF-8">
    <title>Projecto TITANIC</title>
    <style>
        h1 {
            color: #3399FF;
            text-align: center;
        }
        ul {
            list-style-type: none;
        }
        ul li {
            color: lightblue;
        }
        ul ul li {
            color: goldenrod;
        }
        .logo {
            float: right;
            width: 115px;
        }
        .header {
            text-align: center;
        }
    </style>
</head>
<body>
    <div class="header">
        <img src="https://img.icons8.com/officel/80/titanic.png" class="logo" alt="Navio">
        <h1>Projecto TITANIC</h1>
    </div>
    <ul>
        <li>OBJECTIVO:
            <ul>
                <li>Analisar e interpretar quais os tipos de pessoas mais propensas a sobreviver ao desastre do Titanic aplicando ferramentas e estratégias de previsão de quais os passageiros que sobreviveram.</li>
            </ul>
        </li>
        <li>ROADMAP:
            <ul>
                <li>Este plano de trabalho detalhado e estruturado envolverá várias etapas, desde análise exploratória dos dados até à sua modelagem preditiva.</li>
            </ul>
        </li>
        <li>VARIÁVEL TARGET:
            <ul>
                <li>SURVIVED</li>
            </ul>
        </li>
    </ul>
</body>
</html>


# Tarefas do Projeto

Importação das bibliotecas

In [ ]:
import pandas as pd
import numpy as np
np.set_printoptions(precision=2)
import ssl
import certifi
import urllib.request

## 1. Integração e Análise Inicial


### 1.1. Importar o dataset para o ambiente Python e apresentar as primeiras 10 linhas.

In [ ]:
dataset_url = 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv'

ssl_context = ssl.create_default_context(cafile=certifi.where())

with urllib.request.urlopen(dataset_url, context=ssl_context) as response:
    data = pd.read_csv(response)
    
data.head(10)

### 1.2. Apresente um resumo estatístico e uma visão geral do dataset:

In [ ]:
# Número de linhas e colunas
print("Número de linhas e colunas:", data.shape)

# Número de valores ausentes por variável
print("\nValores ausentes por variável:")
print(data.isnull().sum())

# Estatísticas descritivas das variáveis numéricas
print("\nEstatísticas descritivas das variáveis numéricas:")
print(data.describe())

## 2. Limpeza e Transformação de Dados

### 2.1. Trate os valores ausentes:

In [ ]:
# Substitua valores ausentes na variável Age pela mediana. 
data.loc[data['Age'].isnull(), 'Age'] = data['Age'].median()

# Remova registos com valores ausentes em Embarked. 
data.drop(data[data['Embarked'].isnull()].index, inplace=True)

# Substitua valores ausentes em Cabin por "Unknown". 
data.loc[data['Cabin'].isnull(), 'Cabin'] = 'Unknown'

# Dados após o tratamento
print(f'\nDados após o tratamento:\n{data}')

### 2.2. Codifique as variáveis categóricas: 

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# Transforme Sex em valores numéricos (0 para male, 1 para female).
data['Sex'] = data['Sex'].map({'male': 0, 'female': 1})

# Codifique Embarked em variáveis binárias utilizando One-Hot Encoding. 
enc = OneHotEncoder(handle_unknown='ignore', sparse_output=False).set_output(transform='pandas')

result = enc.fit_transform(data[['Embarked']])
print(result)


### 2.3. Crie novas variáveis:

In [ ]:
# FamilySize: Soma de SibSp e Parch.
data['FamilySize'] = data['SibSp'] + data['Parch']

# IsAlone: Variável binária (1 se o passageiro estava sozinho, 0 caso contrário).
data['IsAlone'] = np.where(data['FamilySize'] == 0, 1, 0)

data

## 3. Análise Exploratória e Visualizações

### 3.1. Visualize a distribuição de variáveis como Age, Fare e FamilySize. 
    -   Use histogramas para representar a distribuição.

In [ ]:
data['IsAlone'].value_counts()

In [ ]:
data['Survived'].value_counts()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Histograma para a variável Age
sns.histplot(data['Age'])
plt.title("Distribuição de Age")
plt.figure(figsize=(8, 6))
plt.show()

In [ ]:

# Histograma para a variável Fare
sns.histplot(data['Fare'])
plt.title("Distribuição de Fare")
plt.figure(figsize=(8, 6))
plt.show()

In [ ]:

# Histograma para a variável FamilySize
sns.histplot(data['FamilySize'])
plt.title("Distribuição de FamilySize")
plt.figure(figsize=(8,6))
plt.show()

### 3.2. Analise a relação entre variáveis e a sobrevivência (Survived): 

In [ ]:
# Taxa de sobrevivência por Sex (gráfico de barras)
ax = sns.barplot(data, x='Sex', y='Survived', hue='Sex')
plt.bar_label
for i in ax.containers:
    ax.bar_label(i, padding=-50, fmt='%.2f', color='white', fontweight='bold')

plt.title('Taxa de sobrevivência por Sex')
plt.xlabel('Sex (0 = male, 1 = female)')
plt.figure(figsize=(12, 5))
plt.show()


In [ ]:
# Taxa de sobrevivência por Pclass (gráfico de barras)
ax = sns.barplot(data, x='Pclass', y='Survived', hue='Pclass')
for i in ax.containers:
    ax.bar_label(i, padding=-50, fmt='%.2f', color='white', fontweight='bold')

plt.title('Taxa de sobrevivência por Pclass')
plt.figure(figsize=(12, 5))
plt.show()


In [ ]:

# Distribuição de idades por sobrevivência (histograma com múltiplas cores)
plt.figure(figsize=(8, 6))
sns.histplot(data, x='Age', hue='Survived', multiple='stack')
plt.title('Distribuição de idades por sobrevivência')
plt.xlabel('Survived (0 = Not Survived, 1 = Survived)')
plt.show()

### 3.3. Crie um heatmap para explorar as correlações entre variáveis numéricas. 

In [ ]:
data1 = data[['Survived', 'Pclass', 'Sex', 'Age', 'IsAlone']]
print(data1)
corr = data1.corr()
print(corr)
sns.heatmap(corr, annot=True, fmt=".2f", cmap="RdBu", square=True, vmax=1, vmin=-1)
plt.show()


## 4. Modelação e Comparação de Modelos

### 4.1. Divida os dados em conjuntos de treino e teste. 

In [ ]:
from sklearn.model_selection import train_test_split

# Selecionar as features e a variável alvo.
# Supondo que as colunas 'PassengerId', 'Name', 'Ticket' e 'Cabin' não sejam úteis para a modelagem, elas são removidas do conjunto de features.
features_to_drop = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'Embarked']
X = data.drop(['Survived'] + features_to_drop, axis=1)
y = data['Survived']

# Divisão dos dados: 80% treino e 20% teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print('Dimensão do conjunto de treino: ', {X_train.shape})
print('Dimensão do conjunto de teste: ', {X_test.shape})


### 4.2. Implemente os seguintes modelos preditivos para prever a variável Survived:

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score 
from sklearn.neighbors import KNeighborsClassifier

# Árvore de Decisão
tree_model = DecisionTreeClassifier(random_state=42)
tree_model.fit(X_train, y_train)
tree_pred = tree_model.predict(X_test)

print('Acurácia da Tree: ', accuracy_score(y_test, tree_pred))

# K-Nearest Neighbors (KNN)
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(X_train, y_train)
knn_pred = knn_model.predict(X_test)

print('Acurácia no conjunto de teste: ', accuracy_score(y_test, knn_pred))


### 4.3. Avalie o desempenho dos modelos utilizando as seguintes métricas:

In [ ]:
from sklearn.metrics import confusion_matrix, f1_score

print('Avaliação do modelo Árvore de Decisão:')
print('Matriz de Confusão:\n', confusion_matrix(y_test, tree_pred))
print('Acurácia: ', accuracy_score(y_test, tree_pred))
print('F1-Score:', f1_score(y_test, tree_pred))

print('\nAvaliação do modelo KNN:')
print('Matriz de Confusão:\n', confusion_matrix(y_test, knn_pred))
print('Acurácia: ', accuracy_score(y_test, knn_pred))
print('F1-Score:', f1_score(y_test, knn_pred))


### 4.4. Compare os resultados dos modelos num gráfico ou tabela, identificando o melhor modelo.

In [ ]:
# DataFrame para as métricas dos modelos
metrics = pd.DataFrame({
    'Modelo': ['Árvore de Decisão', 'KNN'],
    'Acurácia': [accuracy_score(y_test, tree_pred), accuracy_score(y_test, knn_pred)],
    'F1-Score': [f1_score(y_test, tree_pred), f1_score(y_test, knn_pred)]
})

# Gráfico de comparação de Acurácia
plt.figure(figsize=(6, 4))
sns.barplot(x='Modelo', y='Acurácia', data=metrics)
plt.title('Comparação da Acurácia')
plt.show()

# Gráfico de comparação de F1-Score
plt.figure(figsize=(6, 4))
sns.barplot(x='Modelo', y='F1-Score', data=metrics)
plt.title('Comparação do F1-Score')
plt.show()

print(metrics)


#### &emsp; O melhor modelo é o da Árvore de decisão, pois apresenta os valores mais aproximados de 1.

## 5. Conclusão

### 5.1. Identifique os grupos de passageiros mais propensos a sobreviver com base na análise exploratória e na importância das variáveis dos modelos.
#### &emsp;-   Explique o impacto de variáveis como Sex, Pclass, Age e IsAlone. 

&emsp;Com base na análise exploratória, de um modo geral, os grupos com maior probabilidade de sobrevivência são Mulheres (Female, ou seja, Sex=1), enquanto o grupo de Homens (Male, ou seja, Sex=0) é o que tem menor probabilidade de sobrevivência.

&emsp;Relativamente às correlações, Sex tem uma forte relação positiva com Survived.<br>
Como observado, a menor taxa de sobrevivência pertence ao grupo Male o que demonstra que, independentemente da correlação, vai influenciar diretamente todas as outras variáveis.

&emsp;Pode-se observar, também, que o grupo da primeira classe (Pclass=1) teve a maior taxa de sobrevivência comparativmente às outras classes.

### 5.2. Justifique o melhor modelo preditivo com base nas métricas de desempenho.

A Árvore de Decisão apresenta uma acurácia e F1-Score superiores aos do KNN, logo, pode-se concluir que esse modelo ajusta-se melhor à presente base de dados.

Ou seja, estas métricas refletem uma melhor capacidade do modelo em prever corretamente os casos de sobrevivência.